# Projeto: análise de um conjunto de dados referente aos tripulantes e passageiros a bordo do Titanic

## Conteúdo
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
    <ul>
        <li><a href="#labels_considerations">Observações sobre os rótulos de colunas</a></li>
    </ul>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>
> If you haven't yet selected and downloaded your data, make sure you do that first before coming back here. If you're not sure what questions to ask right now, then make sure you familiarize yourself with the variables and the dataset context for ideas of what to explore.

In [1]:
# importação de pacotes e carregamento dos dados
import pandas as pd

%matplotlib inline

df = pd.read_csv('titanic-data-6.csv')

<a id='wrangling'></a>
## Data Wrangling

### Propriedades Gerais

In [2]:
#(número de linhas, número de colunas)
df.shape

(891, 12)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


Inicialmente o dataset apresenta 891 linhas e 12 colunas.
<a id="labels_considerations"></a>
#### Observações sobre os rótulos de colunas:

<ul>
    <li>**Survived:** 0 significa que o passageiro em questão não sobreviveu e 1, que o passageiro sobreviveu</li>
    <li>**Pclass:** classe da passage (1 é primeira classe, etc...)</li>
    <li>**SibSp:** número de irmãos / cônjuges a bordo do navio</li>
    <li>**Parch:** número de pais / filhos a bordo do navio</li>
    <li>**Fare:** tarifa paga pelo passageiro</li>
    <li>**Cabin:** número da cabine</li>
    <li>**Embarked:** local de embarque (C = Cherbourg, Q = Queenstown, S = Southampton)</li>
</ul>

*Para mais detalhes consultar <a href="https://www.kaggle.com/c/titanic/data">este link</a>*

In [4]:
#estrutura inicial do dataset
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Data Cleaning

Como se pode notar na seção anterior alguns ajustes precisam ser feitos no dataset para garantir a qualidade da análise.

<a id="column_removal"></a>
#### Remoção de colunas desnecessárias
Os números apresentados na coluna **PassengerId** acompanham o índice automático atribuído pelo *pandas*. Neste caso pode-se optar em transformar a coluna citada em índice ou excluí-la e utilizar o índice automático. A segunda opção foi escolhida.
Para fins desta análise as seguintes colunas serão desconsideradas: PassengerId, Name, SibSp, Parch, Ticket, Fare, Cabin, Embarked. Tal decisão deve-se ao fato de a observação inicial dos dados apontar a irrelevância das colunas desconsideradas na questão da sobrevivência dos passageiros e da tripulação.

In [5]:
#remoção de colunas
cols_to_drop = ['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

df.drop(cols_to_drop, axis=1, inplace=True)

#verificação
df.head()

,Survived,Pclass,Sex,Age
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0


#### Atualização do padrão dos rótulos de coluna
Para facilitar a legibilidade os rótulos serão atualizados de forma a apresentarem apenas letras minúsculas

In [6]:
df.rename(columns = lambda x: x.lower(), inplace=True)

df.head()

,survived,pclass,sex,age
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0


#### Tratamento de colunas com valores nulos
Ao se verificar a estrutura atual do dataframe pode-se notar que a coluna **age** apresenta 177 valores nulos. Como essas linhas representam 19,86% do dataframe seria imprudente descartá-las da análise. Para corrigir o problema os valores nulos serão preenchidos com a média das idades apresentadas na coluna citada. 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
survived    891 non-null int64
pclass      891 non-null int64
sex         891 non-null object
age         714 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 27.9+ KB


In [8]:
df['age'].isnull().sum()

177

In [9]:
#cálculo da média das idades
age_mean = df['age'].mean()

#preenchimento dos valores vazios na coluna "age"
df['age'].fillna(age_mean, inplace=True)

#verificação da operação
df['age'].isnull().sum()

0

#### Mudanças de tipo de dados

Os dados da coluna **survived** são do tipo *int* e os da coluna **age**, *float*. Para melhorar a legibidade dos dados será usado o tipo *bool* para **survived** e *int* para **age**

In [10]:
#conversão da coluna survived
df['survived'] = df['survived'].astype(bool)

#conversão da coluna age
df['age'] = df['age'].astype(int)

In [11]:
df.head()

,survived,pclass,sex,age
0,False,3,male,22
1,True,1,female,38
2,True,3,female,26
3,True,1,female,35
4,False,3,male,35


<a id='eda'></a>
## Exploratory Data Analysis
<a id="question_1"></a>
### Questão 1: qual a relação entre a idade e o sexo entre os sobreviventes e o não-sobreviventes? 


In [12]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Questão 2: qual a relação entre a classe do bilhete entre os sobreviventes e os não-sobreviventes?

In [13]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!